In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
data = np.loadtxt(open("../data/Vela_Flux.txt", 'rb'), usecols=range(7))
# If for some reason that doesn't work, then you can download that file to your jupyter area and do:
# data = np.loadtxt(open("Hubble.txt", 'rb'), usecols=[1,2,3])
# This is how we pull out the data from columns in the array.
date_MET = data[:,0]
nObs_LE = data[:,1]
nExp_LE = data[:,2]
signif_LE = data[:3]
nObs_HE = data[:,4]
nExp_HE = data[:,5]
signif_HE = data[:6]
LE_bounds = (100., 800.)
HE_bounds = (800., 10000.)



In [ ]:
1./np.sqrt(np.mean(nObs_HE))